# CBAM Risk Analysis: East Asia & Pacific

## Carbon Border Adjustment Mechanism Impact Assessment

This notebook analyzes the vulnerability of East Asian and Pacific countries to the EU's Carbon Border Adjustment Mechanism (CBAM). The analysis combines multiple dimensions:

- **Exposure**: CBAM exports relative to GDP and total exports
- **Vulnerability**: Carbon intensity of electricity mix
- **Adaptive Capacity**: Statistical capacity and innovation capability
- **Economic Flexibility**: Trade elasticity

---

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add src directory to path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root / 'src'))

# Import custom modules
from data_loader import DataLoader
from data_processing import DataProcessor
from visualization import Visualizer
from analysis import RiskAnalyzer
from config import RESULTS_DIR

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✓ All modules imported successfully")
print(f"✓ Project root: {project_root}")

## 2. Load Data

Load all required datasets using the DataLoader class.

In [ ]:
# Initialize data loader
loader = DataLoader()

# Load all datasets
print("Loading datasets...\n")
raw_data = loader.load_all()

print(f"\n✓ Loaded {len(raw_data)} datasets successfully")
print("\nDataset shapes:")
for name, df in raw_data.items():
    print(f"  {name:20s}: {df.shape}")

## 3. Process Data

Clean, transform, and prepare all datasets for analysis.

In [ ]:
# Initialize processor
processor = DataProcessor()

# Process each dataset
print("Processing datasets...\n")

processed_data = {
    'cbam_exports': processor.prepare_cbam_exports(raw_data['cbam_exports']),
    'total_exports': processor.prepare_total_exports(raw_data['total_exports']),
    'gdp': raw_data['gdp'],
    'carbon_intensity': processor.prepare_carbon_intensity(raw_data['carbon_intensity']),
    'spi': processor.prepare_spi(raw_data['spi']),
    'patents': processor.prepare_patents(raw_data['patents']),
    'population': processor.prepare_population(raw_data['population']),
    'trade_elasticity': processor.prepare_trade_elasticity(raw_data['trade_elasticity'])
}

print("\n✓ All datasets processed successfully")

## 4. Exploratory Data Analysis

### 4.1 CBAM Exports Analysis

In [ ]:
# Initialize visualizer
viz = Visualizer()

# Display top CBAM exporters
print("Top 10 CBAM Exporters to EU (2019):")
print(processed_data['cbam_exports'][['ReporterISO', 'CBAM Value (millions)']].head(10))

# Visualize
viz.create_bar_chart(
    processed_data['cbam_exports'].head(15),
    x_col='ReporterISO',
    y_col='CBAM Value (millions)',
    title='CBAM Export Value by Country (Top 15)',
    ylabel='Export Value (millions USD)'
)
plt.show()

### 4.2 CBAM Exports as Proportion of GDP

In [ ]:
# Merge CBAM with GDP
cbam_gdp = processor.merge_cbam_gdp(
    processed_data['cbam_exports'],
    processed_data['gdp']
)

print("CBAM Exports as % of GDP:")
print(cbam_gdp[['ReporterISO', 'ExpCBAMperGDP']].head(10))

viz.create_bar_chart(
    cbam_gdp,
    x_col='ReporterISO',
    y_col='ExpCBAMperGDP',
    title='CBAM Exports as Proportion of GDP',
    ylabel='Proportion of GDP',
    sort_by='ExpCBAMperGDP',
    ascending=False
)
plt.show()

**Key Insight**: Countries where CBAM exports represent a higher proportion of GDP face greater economic exposure to carbon pricing mechanisms.

### 4.3 CBAM Exports as Proportion of Total Exports

In [ ]:
# Merge total exports with CBAM
cbam_total = processor.merge_cbam_total_exports(
    processed_data['total_exports'],
    processed_data['cbam_exports']
)

print("CBAM Exports as % of Total Exports:")
print(cbam_total[['ReporterISO', 'PctExpCBAM']].head(10))

viz.create_bar_chart(
    cbam_total,
    x_col='ReporterISO',
    y_col='PctExpCBAM',
    title='CBAM Exports as Proportion of Total Exports',
    ylabel='Proportion of Total Exports',
    sort_by='PctExpCBAM',
    ascending=False
)
plt.show()

### 4.4 Two-Dimensional Exposure Analysis

In [ ]:
# Merge both exposure metrics
two_dim = pd.merge(cbam_total, cbam_gdp, on='ReporterISO', how='left')
two_dim = two_dim[['ReporterISO', 'PctExpCBAM', 'ExpCBAMperGDP']].dropna()

viz.create_scatter_plot(
    two_dim,
    x_col='PctExpCBAM',
    y_col='ExpCBAMperGDP',
    label_col='ReporterISO',
    title='Two-Dimensional Exposure Analysis',
    xlabel='CBAM Exports / Total Exports',
    ylabel='CBAM Exports / GDP'
)
plt.show()

### 4.5 Carbon Intensity Analysis

In [ ]:
print("Carbon Intensity of Electricity (gCO2e/kWh):")
print(processed_data['carbon_intensity'][['Country Code', 'Carbon_Intensity']].head(10))

viz.create_bar_chart(
    processed_data['carbon_intensity'],
    x_col='Country Code',
    y_col='Carbon_Intensity',
    title='Carbon Intensity of Electricity Mix',
    ylabel='Carbon Intensity (gCO2e/kWh)',
    sort_by='Carbon_Intensity',
    ascending=False
)
plt.show()

**Key Insight**: Higher carbon intensity indicates greater vulnerability to carbon border adjustments.

## 5. Build Comprehensive Dataset

Merge all dimensions into a single comprehensive dataset.

In [ ]:
# Build comprehensive dataset
comprehensive_df = processor.build_comprehensive_dataset(processed_data)

print("Comprehensive Dataset:")
print(f"Shape: {comprehensive_df.shape}")
print(f"\nColumns: {list(comprehensive_df.columns)}")
print(f"\nSample data:")
print(comprehensive_df[['Country Name', 'ExpCBAMperGDP', 'PctExpCBAM', 
                        'Carbon_Intensity', 'SPI_Score', 'PatPerCap', 'Trade_Elast']].head())

## 6. Normalize and Prepare for Analysis

In [ ]:
# Normalize columns for analysis
analysis_df = processor.normalize_columns(comprehensive_df)

# Create complementary scores (lower values = worse, so we invert)
analysis_df = processor.create_complementary_scores(
    analysis_df,
    columns=['SPI_Score', 'PatPerCap']
)

# Drop original columns, keep complementary
analysis_df = analysis_df.drop(columns=['SPI_Score', 'PatPerCap'], errors='ignore')

print("Normalized dataset ready for risk analysis")
print(f"Shape: {analysis_df.shape}")
print(f"\nNormalized columns sample:")
print(analysis_df[['Country Name', 'ExpCBAMperGDP', 'Carbon_Intensity', 
                   'SPI_Score_Compl', 'PatPerCap_Compl']].head())

## 7. Risk Index Calculation

### 7.1 Baseline Risk Index

In [ ]:
# Initialize risk analyzer
analyzer = RiskAnalyzer()

# Calculate baseline risk index
from config import WEIGHT_SCENARIOS

baseline_weights = WEIGHT_SCENARIOS['baseline']
print("Baseline weights:")
for component, weight in baseline_weights.items():
    print(f"  {component:20s}: {weight:.2%}")

# Calculate weighted index
analysis_df = analyzer.calculate_weighted_index(
    analysis_df,
    baseline_weights,
    index_name='CBAM_Risk_Index'
)

# Rank countries
results_df = analyzer.rank_countries(
    analysis_df,
    index_col='CBAM_Risk_Index',
    country_col='Country Name'
)

print("\n" + "="*60)
print("CBAM RISK INDEX RANKINGS")
print("="*60)
print(results_df[['Rank', 'Country Name', 'CBAM_Risk_Index']].head(15))

### 7.2 Visualize Risk Index

In [ ]:
viz.create_risk_index_chart(
    results_df,
    country_col='Country Name',
    index_col='CBAM_Risk_Index',
    save_path=RESULTS_DIR / 'cbam_risk_index.png'
)
plt.show()

### 7.3 Summary Statistics

In [ ]:
# Get summary statistics
stats = analyzer.get_summary_statistics(results_df, 'CBAM_Risk_Index')
print("\nRisk Index Statistics:")
print(stats)

### 7.4 Risk Categorization

In [ ]:
# Categorize countries by risk level
results_df = analyzer.categorize_risk(results_df, 'CBAM_Risk_Index')

print("\nCountries by Risk Category:")
print(results_df.groupby('RiskCategory')['Country Name'].apply(list))

print("\nRisk Category Distribution:")
print(results_df['RiskCategory'].value_counts())

## 8. Sensitivity Analysis

Test how different weighting schemes affect the rankings.

In [ ]:
# Run sensitivity analysis with all scenarios
sensitivity_df = analyzer.run_sensitivity_analysis(analysis_df)

# Display available scenarios
scenario_cols = [col for col in sensitivity_df.columns if col.startswith('Index_')]
print(f"Generated {len(scenario_cols)} scenarios:")
for col in scenario_cols:
    scenario_name = col.replace('Index_', '')
    print(f"  - {scenario_name}")

### 8.1 Compare Scenario Rankings

In [ ]:
# Compare top countries across scenarios
comparison = analyzer.compare_scenarios(
    sensitivity_df,
    scenario_cols=scenario_cols,
    country_col='Country Name',
    top_n=10
)

print("\nTop 10 Countries - Ranking Comparison Across Scenarios:")
print(comparison)

### 8.2 Visualize Sensitivity Analysis

In [ ]:
# Create scenario names for display
scenario_names = [col.replace('Index_', '').replace('_', ' ').title() 
                 for col in scenario_cols]

viz.create_sensitivity_comparison(
    sensitivity_df,
    country_col='Country Name',
    scenario_cols=scenario_cols,
    scenario_names=scenario_names,
    save_path=RESULTS_DIR / 'sensitivity_analysis.png'
)
plt.show()

## 9. Risk Driver Analysis

Identify the main risk factors for selected countries.

In [ ]:
# Select top 5 highest risk countries
top_countries = results_df.nsmallest(5, 'Rank')['Country Name'].tolist()

print("Risk Drivers for Top 5 Highest Risk Countries:\n")
print("="*60)

for country in top_countries:
    print(f"\n{country}:")
    print("-" * 40)
    drivers = analyzer.identify_risk_drivers(
        analysis_df,
        country=country,
        component_cols=['ExpCBAMperGDP', 'PctExpCBAM', 'Carbon_Intensity',
                       'Trade_Elast', 'SPI_Score_Compl', 'PatPerCap_Compl']
    )
    for component, value in drivers.items():
        print(f"  {component:20s}: {value:.4f}")

## 10. Generate Comprehensive Report

In [ ]:
# Generate comprehensive risk report
report = analyzer.generate_risk_report(
    results_df,
    index_col='CBAM_Risk_Index',
    country_col='Country Name',
    top_n=5,
    bottom_n=5
)

print("\n" + "="*60)
print("CBAM RISK ASSESSMENT - COMPREHENSIVE REPORT")
print("="*60)

print(f"\nTotal countries analyzed: {report['total_countries']}")
print(f"Mean risk index: {report['mean_risk']:.4f}")
print(f"Median risk index: {report['median_risk']:.4f}")

print("\n5 HIGHEST RISK COUNTRIES:")
print(report['highest_risk'].to_string(index=False))

print("\n5 LOWEST RISK COUNTRIES:")
print(report['lowest_risk'].to_string(index=False))

## 11. Export Results

In [ ]:
# Export main results
output_path = RESULTS_DIR / 'cbam_risk_analysis_results.xlsx'
analyzer.export_results(
    results_df[['Rank', 'Country Name', 'Country Code', 'CBAM_Risk_Index', 'RiskCategory',
               'ExpCBAMperGDP', 'PctExpCBAM', 'Carbon_Intensity', 
               'Trade_Elast', 'SPI_Score_Compl', 'PatPerCap_Compl']],
    output_path=output_path,
    sheet_name='Risk Analysis'
)

print(f"✓ Results exported to: {output_path}")

# Export sensitivity analysis
sensitivity_output = RESULTS_DIR / 'sensitivity_analysis_results.xlsx'
analyzer.export_results(
    sensitivity_df,
    output_path=sensitivity_output,
    sheet_name='Sensitivity Analysis'
)

print(f"✓ Sensitivity analysis exported to: {sensitivity_output}")

## 12. Key Findings and Conclusions

### Highest Risk Countries
Based on the comprehensive risk index, the countries most vulnerable to CBAM impacts are those with:
- High proportion of CBAM-covered exports relative to GDP and total exports
- High carbon intensity in electricity production
- Lower statistical and innovation capacity

### Lowest Risk Countries
Countries with lower risk profiles typically have:
- Lower exposure through CBAM-covered exports
- Cleaner electricity mix (lower carbon intensity)
- Higher adaptive capacity (better statistics and innovation)

### Policy Implications
1. **High-risk countries** should prioritize:
   - Decarbonization of electricity generation
   - Export diversification strategies
   - Strengthening statistical systems for carbon reporting
   - Investing in green technologies and innovation

2. **Medium-risk countries** should:
   - Monitor CBAM implementation closely
   - Begin transition planning for cleaner production
   - Build capacity for carbon accounting

3. **Regional cooperation** opportunities:
   - Knowledge sharing on carbon reduction
   - Joint capacity building initiatives
   - Coordinated policy responses

---

**Note**: This analysis uses 2019 data. Results should be updated with latest available data for current policy decisions.